In [1]:
!pip install pyspark

You should consider upgrading via the '/home/fmenegolo/.asdf/installs/python/3.9.1/bin/python3.9 -m pip install --upgrade pip' command.
Reshimming asdf python...


# Perguntas dessa tarefa

## 1.

- Inicialmente iremos preparar o ambiente, definindo o diretório onde nosso código será desenvolvido. Para este diretório iremos copiar o arquivo nomes_aleatorios.txt.

- Após, em nosso script Python, devemos importar as bibliotecas necessárias:

from pyspark.sql import SparkSession

from pyspark import SparkContext, SQLContext

- Aplicando as bibliotecas do Spark, podemos definir a Spark Session e sobre ela definir o Context para habilitar o módulo SQL

spark = SparkSession \

                .builder \

                .master("local[*]")\

                .appName("Exercicio Intro") \

                .getOrCreate()



In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.functions import col, substring, when, size
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, DateType, LongType, ArrayType

spark = SparkSession \
    .builder \
    .master("local[*]")\
    .appName("ELT TRUSTED") \
    .getOrCreate()


/usr/sbin/hadoop


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/apache-spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/26 15:49:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/26 15:49:58 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# Definir o esquema para o DataFrame df_movies_IMDB
schema_movies_IMDB = StructType([
    StructField("id", StringType(), True),
    StructField("tituloPincipal", StringType(), True),
    StructField("tituloOriginal", StringType(), True),
    StructField("anoLancamento", StringType(), True),
    StructField("tempoMinutos", StringType(), True),
    StructField("genero", StringType(), True),
    StructField("notaMedia", FloatType(), True),  # Definir tipo como FloatType()
    StructField("numeroVotos", IntegerType(), True),  # Definir tipo como IntegerType()
    StructField("generoArtista", StringType(), True),
    StructField("personagem", StringType(), True),
    StructField("nomeArtista", StringType(), True),
    StructField("anoNascimento", StringType(), True),  
    StructField("anoFalecimento", StringType(), True),  
    StructField("profissao", StringType(), True),
    StructField("titulosMaisConhecidos", StringType(), True)
])

# Definir o esquema para o DataFrame df_series_IMDB
schema_series_IMDB = StructType([
    StructField("id", StringType(), True),
    StructField("tituloPincipal", StringType(), True),
    StructField("tituloOriginal", StringType(), True),
    StructField("anoLancamento", StringType(), True), 
    StructField("anoTermino", StringType(), True),  
    StructField("tempoMinutos", IntegerType(), True),  # Definir tipo como IntegerType()
    StructField("genero", StringType(), True),
    StructField("notaMedia", FloatType(), True),  # Definir tipo como FloatType()
    StructField("numeroVotos", IntegerType(), True),  # Definir tipo como IntegerType()
    StructField("generoArtista", StringType(), True),
    StructField("personagem", StringType(), True),
    StructField("nomeArtista", StringType(), True),
    StructField("anoNascimento", StringType(), True),  
    StructField("anoFalecimento", StringType(), True),  
    StructField("profissao", StringType(), True),
    StructField("titulosMaisConhecidos", StringType(), True)
])

- Nesta etapa, é a leitura dos arquivos de filmes e series CSV e JSON

In [4]:
path1 = f"/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_7/Desafio01_ETL/assets/"
df_movies_IMDB = spark.read.option("delimiter", "|").schema(schema_movies_IMDB).csv(path1 + 'movies.csv', header=True)
df_series_IMDB = spark.read.option("delimiter", "|").schema(schema_series_IMDB).csv(path1 + 'series.csv', header=True)

path2 = f"/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_8/Desafio02_ETL/Tarefa_02/assets/"
df_movies_TMDB = spark.read.json(path2 + 'movies.json') 
df_series_TMDB = spark.read.json(path2 + 'series.json')

df_movies_IMDB.show(1)
df_series_IMDB.show(1)

df_movies_TMDB.show(1)
df_series_TMDB.show(1)

+---------+--------------+--------------+-------------+------------+-------+---------+-----------+-------------+--------------------+---------------+-------------+--------------+---------+---------------------+
|       id|tituloPincipal|tituloOriginal|anoLancamento|tempoMinutos| genero|notaMedia|numeroVotos|generoArtista|          personagem|    nomeArtista|anoNascimento|anoFalecimento|profissao|titulosMaisConhecidos|
+---------+--------------+--------------+-------------+------------+-------+---------+-----------+-------------+--------------------+---------------+-------------+--------------+---------+---------------------+
|tt0000009|    Miss Jerry|    Miss Jerry|         1894|          45|Romance|      5.3|        200|      actress|Miss Geraldine Ho...|Blanche Bayliss|         1878|          1951|  actress|            tt0000009|
+---------+--------------+--------------+-------------+------------+-------+---------+-----------+-------------+--------------------+---------------+-------

## 2. 
 Filtro os Filmes do BD Local com lista de filmes do TMDB

In [11]:
# Converter a coluna 'title' do DataFrame JSON para um array de palavras e extrair o ano de lançamento
df_filmes_titulos_datas = df_movies_TMDB.select(col('title'), substring(col('release_date'), 1, 4).alias('releaseDate'))

# Realizar a filtragem dos títulos
df_IMDB_filtrado_filmes = df_movies_IMDB.join(df_filmes_titulos_datas,
                                   (df_movies_IMDB['tituloPincipal'].contains(df_filmes_titulos_datas['title'])) &
                                   (df_movies_IMDB['anoLancamento'] == df_filmes_titulos_datas['releaseDate']),
                                   'inner')

# Remover as colunas 'title' e 'releaseDate' do DataFrame
df_IMDB_filtrado_filmes = df_IMDB_filtrado_filmes.drop('releaseDate')

# Renomear a coluna 'tituloPincipal' para 'tituloPrincipal' no DataFrame df_IMDB_filtrado_filmes
df_IMDB_filtrado_filmes = df_IMDB_filtrado_filmes.withColumnRenamed("tituloPincipal", "tituloPrincipal")

df_IMDB_filtrado_filmes = df_IMDB_filtrado_filmes.distinct()

# Trata os valores "NA" na coluna "anoFalecimento"
df_IMDB_Filmes_resultado = df_IMDB_filtrado_filmes.withColumn("anoFalecimento", when(col("anoFalecimento") == "\\N", None).otherwise(col("anoFalecimento")))

# Exibir resultado
df_IMDB_Filmes_resultado.show(20)
print("Número de linhas antes IMDB: ", df_movies_IMDB.count())
print("Número de linhas depois IMDB: ", df_IMDB_Filmes_resultado.count())
print("Número de linhas TMDB: ", df_movies_TMDB.count())
count = df_IMDB_Filmes_resultado.select('title').distinct().count()
print("Número de linhas no resultado: ", count)

+----------+--------------------+--------------------+-------------+------------+--------------------+---------+-----------+-------------+--------------------+--------------------+-------------+--------------+--------------------+---------------------+--------------------+
|        id|     tituloPrincipal|      tituloOriginal|anoLancamento|tempoMinutos|              genero|notaMedia|numeroVotos|generoArtista|          personagem|         nomeArtista|anoNascimento|anoFalecimento|           profissao|titulosMaisConhecidos|               title|
+----------+--------------------+--------------------+-------------+------------+--------------------+---------+-----------+-------------+--------------------+--------------------+-------------+--------------+--------------------+---------------------+--------------------+
| tt0811080|         Speed Racer|         Speed Racer|         2008|         135|Action,Adventure,...|      6.0|      74455|        actor|             Racer X|         Matthew Fo

Número de linhas antes IMDB:  1045161


Número de linhas depois IMDB:  66
Número de linhas TMDB:  49


Número de linhas no resultado:  14


## 3. 
Filtro as Series do BD Local com lista de series do TMDB

In [13]:
# Converter a coluna 'name' e a substring de 'first_air_date' do DataFrame JSON para o DataFrame do TMDB
df_series_nomes_datas = df_series_TMDB.select(col('name'), substring(col('first_air_date'), 1, 4).alias('dataLancamento'))

# Realizar a filtragem dos títulos e anos de lançamento
df_IMDB_filtrado_series = df_series_IMDB.join(df_series_nomes_datas,
                                         (df_series_IMDB['tituloPincipal'].contains(df_series_nomes_datas['name'])) &
                                         (df_series_IMDB['anoLancamento'] == df_series_nomes_datas['dataLancamento']),
                                         'inner')

# Remover as colunas 'name' e 'dataLancamento' do DataFrame
#df_IMDB_filtrado_series = df_IMDB_filtrado_series.drop('name','dataLancamento')

df_IMDB_filtrado_series = df_IMDB_filtrado_series.distinct()

# Renomear a coluna 'tituloPincipal' para 'tituloPrincipal' no DataFrame df_IMDB_filtrado_filmes
df_IMDB_filtrado_series = df_IMDB_filtrado_series.withColumnRenamed("tituloPincipal", "tituloPrincipal")

# Exibir resultado
df_IMDB_filtrado_series.show(10)
print("Número de linhas antes IMDB: ", df_series_IMDB.count())
print("Número de linhas IMDB: ", df_IMDB_filtrado_series.count())
print("Número de linhas TMDB: ", df_series_TMDB.count())

count = df_IMDB_filtrado_series.select('name').distinct().count()
print("Número de linhas no resultado: ", count)

+---------+--------------------+--------------------+-------------+----------+------------+--------------------+---------+-----------+-------------+--------------------+-----------------+-------------+--------------+--------------------+---------------------+--------------------+--------------+
|       id|     tituloPrincipal|      tituloOriginal|anoLancamento|anoTermino|tempoMinutos|              genero|notaMedia|numeroVotos|generoArtista|          personagem|      nomeArtista|anoNascimento|anoFalecimento|           profissao|titulosMaisConhecidos|                name|dataLancamento|
+---------+--------------------+--------------------+-------------+----------+------------+--------------------+---------+-----------+-------------+--------------------+-----------------+-------------+--------------+--------------------+---------------------+--------------------+--------------+
|tt0094452|        Double First|        Double First|         1988|        \N|          60|              Comedy|

Número de linhas IMDB:  22271
Número de linhas TMDB:  10000


Número de linhas no resultado:  2298


## 4. 
Filtro as Series do BD Local que foram filtradas no passo 3

com a dos filmes do BD local que foram filtrados no passo 2

In [7]:
"""# Filtrar apenas os títulos das séries correspondentes aos filmes
df_series_correspondentes = df_IMDB_Filmes_resultado.select(col('title').alias('titulo'))"""

df_series_correspondentes = df_movies_TMDB.select(col('title').alias('titulo'))

"""# Realizar a junção entre os dataframes de séries e títulos correspondentes usando contains
df_IMDB_Series_resultado = df_IMDB_filtrado_series.join(df_series_correspondentes,
                                            df_IMDB_filtrado_series['tituloPrincipal'].contains(df_series_correspondentes['titulo']),
                                            'inner')"""
df_IMDB_Series_resultado = df_IMDB_filtrado_series.join(df_series_correspondentes,
                                                        df_IMDB_filtrado_series['tituloPrincipal'].contains(df_series_correspondentes['titulo']),
                                                        'inner')


df_IMDB_Series_resultado = df_IMDB_Series_resultado.drop('titulo')

# Remover linhas duplicadas no resultado
df_IMDB_Series_resultado = df_IMDB_Series_resultado.distinct()

# Trata os valores "NA" na coluna "anoFalecimento"
df_IMDB_filtrado_series = df_IMDB_filtrado_series.withColumn("anoNascimento", when(col("anoNascimento") == "\\N", None).otherwise(col("anoNascimento")))
df_IMDB_filtrado_series = df_IMDB_filtrado_series.withColumn("anoFalecimento", when(col("anoFalecimento") == "\\N", None).otherwise(col("anoFalecimento")))
df_IMDB_filtrado_series = df_IMDB_filtrado_series.withColumn("anoTermino", when(col("anoTermino") == "\\N", None).otherwise(col("anoTermino")))
df_IMDB_filtrado_series = df_IMDB_filtrado_series.withColumn("tempoMinutos", when(col("tempoMinutos") == "\\N", None).otherwise(col("tempoMinutos")))

# Exibir resultado das séries correspondentes aos filmes
df_IMDB_Series_resultado.show(10)

print("Número de linhas IMDB antes: ", df_IMDB_filtrado_series.count())
print("Número de linhas no resultado: ", df_IMDB_Series_resultado.count())

+---------+--------------------+--------------------+-------------+----------+------------+--------------------+---------+-----------+-------------+--------------------+--------------+-------------+--------------+--------------------+---------------------+
|       id|     tituloPrincipal|      tituloOriginal|anoLancamento|anoTermino|tempoMinutos|              genero|notaMedia|numeroVotos|generoArtista|          personagem|   nomeArtista|anoNascimento|anoFalecimento|           profissao|titulosMaisConhecidos|
+---------+--------------------+--------------------+-------------+----------+------------+--------------------+---------+-----------+-------------+--------------------+--------------+-------------+--------------+--------------------+---------------------+
|tt0061300|         Speed Racer|        Mahha GoGoGo|         1967|      1968|          30|Action,Adventure,...|      7.2|       3041|        actor|Pops RacerAdditio...|   Jack Curtis|         1926|          1970|actor,writer,dir

Número de linhas IMDB antes:  22143


Número de linhas no resultado:  100


## 5. 
Filtro as Series do TMDB com lista de Filmes do TMDB

In [8]:
# Filtrar apenas os títulos das séries correspondentes aos filmes
df_filmes_titulos = df_movies_TMDB.select(col('title').alias('titulo'))

# Realizar a junção entre os dataframes de séries e títulos correspondentes usando contains
df_TMDB_Series_resultado = df_series_TMDB.join(df_filmes_titulos,
                                            df_series_TMDB['name'].contains(df_filmes_titulos['titulo']),
                                            'inner')

df_TMDB_Series_resultado = df_TMDB_Series_resultado.drop('titulo','backdrop_path','poster_path')

# Remover linhas duplicadas no resultado
df_TMDB_Series_resultado = df_TMDB_Series_resultado.distinct()

# Trata os valores "NA" na coluna "anoFalecimento"
df_TMDB_Series_resultado = df_TMDB_Series_resultado.withColumn("first_air_date", when(col("first_air_date") == "", None).otherwise(col("first_air_date")))
df_TMDB_Series_resultado = df_TMDB_Series_resultado.withColumn("genre_ids", when(size(col("genre_ids")) == 0, None).otherwise(col("genre_ids")))
df_TMDB_Series_resultado = df_TMDB_Series_resultado.withColumn("overview", when(col("overview") == "", None).otherwise(col("overview")))

# Exibir resultado das séries correspondentes aos filmes
df_TMDB_Series_resultado.show(10)
print("Número de linhas antes TMDB: ", df_series_TMDB.count())
print("Número de linhas no resultado: ", df_TMDB_Series_resultado.count())

+--------------+--------------------+------+--------------------+--------------+-----------------+-----------------------------+--------------------+----------+------------+----------+
|first_air_date|           genre_ids|    id|                name|origin_country|original_language|                original_name|            overview|popularity|vote_average|vote_count|
+--------------+--------------------+------+--------------------+--------------+-----------------+-----------------------------+--------------------+----------+------------+----------+
|    1967-09-23|[10759, 16, 35, 1...| 30986|         Speed Racer|          [JP]|               ja|                 マッハGoGoGo|Teenager Speed Ra...|    29.117|         7.4|        97|
|          null|                [16]|222624|Gintama 3-nen Z-g...|          [JP]|               ja|          銀魂 3年Z組銀八先生|The storyline rev...|       0.6|         0.0|         0|
|    2022-10-04|         [16, 10759]|158293|   Shinobi no Ittoki|          [JP]|      

## 6.
Elimina algumas colunas não relevantes do df_movies_TMDB 

In [9]:
df_TMDB_filmes_resultado = df_movies_TMDB.drop('adult','backdrop_path','poster_path','video')

df_movies_TMDB.show(5)
df_TMDB_filmes_resultado.show(5)

+-----+--------------------+-------------+------+-----------------+--------------------------------+--------------------+----------+--------------------+------------+--------------------+-----+------------+----------+
|adult|       backdrop_path|    genre_ids|    id|original_language|                  original_title|            overview|popularity|         poster_path|release_date|               title|video|vote_average|vote_count|
+-----+--------------------+-------------+------+-----------------+--------------------------------+--------------------+----------+--------------------+------------+--------------------+-----+------------+----------+
|false|/kgATFkG4SDyengNM...| [14, 28, 12]|455476|               en|            Knights of the Zo...|Quando um órfão d...|   292.303|/omPLRVEydDJdkXoB...|  2023-04-27|Os Cavaleiros do ...|false|         6.3|        49|
|false|/5hoS3nEkGGXUfmnu...| [28, 12, 14]|960704|               ja|  鋼の錬金術師 完結編 最後の錬成|Acompanhe a últim...|    82.899|/uMzOYHGiH

## 6.

In [10]:
"""
# Salvar df_IMDB_Filmes_resultado em formato Parquet
df_IMDB_Filmes_resultado.write.csv("/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_9/Desafio03_ETL/Tarefa_03/salvar_csv/df_IMDB_filtrado_filmes.csv")

# Salvar df_IMDB_Series_resultado em formato Parquet
df_IMDB_Series_resultado.write.csv("/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_9/Desafio03_ETL/Tarefa_03/salvar_csv/df_IMDB_filtrado_series.csv")

# Salvar df_TMDB_Series_resultado em formato Parquet
df_TMDB_Series_resultado.write.csv("/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_9/Desafio03_ETL/Tarefa_03/salvar_csv/df_IMDB_Series_resultado.scv")

# Salvar df_df_TMDB_filmes_resultado em formato Parquet
df_TMDB_filmes_resultado.write.csv("/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_9/Desafio03_ETL/Tarefa_03/salvar_csv/df_TMDB_Series_resultado.csv")
"""


#df_IMDB_Series_resultado.write.csv("/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_9/Desafio03_ETL/Tarefa_03/salvar_csv/df_TMDB_Series_resultado2.csv")


ERROR:root:KeyboardInterrupt while sending command.==>              (6 + 2) / 8]
Traceback (most recent call last):
  File "/home/fmenegolo/.asdf/installs/python/3.9.1/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/fmenegolo/.asdf/installs/python/3.9.1/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/fmenegolo/.asdf/installs/python/3.9.1/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 